# Homework 8

## Problem 1:  Square Roots Modulo $p$

In this problem you will implement the algorithm for computing square roots modulo an odd prime $p$, as [described in the book](https://luisfinotti.org/pcimc/13-Square_Roots.html#square-roots-module-odd-primes).  You can use Sage's `is_square` function to determine if your element has a square root (i.e., is a square).  If not, your function should return `None`.  (This is already done for you below.)

The input is simply some $a \in \mathbb{F}_p$ (for some $p$), and the output should be some $b \in \mathbb{F}_p$ such that $b^2 = a$ (in $\mathbb{F}_p$, if exists).  (Note that $b$ is not unique unless $a=0$.  But the only other possibility is $-b$.)

**Note:** 

- Again, you can get $\mathbb{F}_p$ from $a$ with `F = parent(a)` and $p$ from $\mathbb{F}_p$ with `p = F.characteristic()`.
- You hould do the cases of $p \equiv 1  \pmod{4}$ and $p \equiv 3 \pmod{4}$ separately.  For the former, use the [Tonelli-Shanks Algorithm](https://luisfinotti.org/pcimc/13-Square_Roots.html#al-TS).
- You should also take care of the cases $a=0$ and $a=1$ separately.

In [ ]:
def sqrt_modp(a):
    """
    Given a in F_p, computes one of its square roots if possible, or returns
    None if there is none.

    INPUT:
    a: an element of FiniteField(p).

    OUTPUT:
    A square root of a (in F_p), if it exists, and None if it does not.
    """
    if not is_square(a):
        return None
    ...

Let's test it:

In [ ]:
number_of_tests = 200
min_p = 1000
max_p = 2000

for _ in range(number_of_tests):
    p = random_prime(max_p, lbound = min_p)
    F = FiniteField(p)
    a = F(randint(0, p - 1))
    result = sqrt_modp(a)
    if is_square(a):
        if result == None:
            print(f"Your function gave that {a} mod {p} has no square root, which is incorrect.")
            break
        else:
            if result^2 != a:
                print(f"Your function gave {result} as a square root of {a} mod {p}, which is incorrect.")
                break
    else:
        if result != None:
            print(f"Your function gave {result} as a square root of {a} mod {p}, but there is no square root in this case.")
            break
else:
    print("It works!")

## Problem 2: Square Roots Module $p^n$

Given an *odd* prime $p$, a power $n \geq 2$, and integers $a$ and $b$ with $p \nmid a$ and $b^2 \equiv a \pmod{p}$, you will use *Hensel's Lemma*, more precisely, [its application to square roots](https://luisfinotti.org/pcimc/13-Square_Roots.html#th-hl-sqrt-odd) to find some integer $x$ such that $x^2 \equiv a \pmod{p^n}$.

**We will skip the case $p=2$.**  (It is [done in the book](https://luisfinotti.org/pcimc/13-Square_Roots.html#implementation).)

We will assume that every input is an integer.  Your (unique) solution should be also an *integer* in $\{0, 1, \ldots, (p^n-1)\}$ (i.e., reduced modulo $p^n$).

(You do *not* need to check that $p \nmid a$ or $b^2 \equiv a \pmod{p}$.)

**Hint:** You just need to follow the [algorithm in the book](https://luisfinotti.org/pcimc/13-Square_Roots.html#al-sqrt-power-p).  Also, see the [example done in the book](https://luisfinotti.org/pcimc/13-Square_Roots.html#id3).

In [ ]:
def my_sqrt(a, b, p, n):
    """
    Given b such that b^2 is a modulo p, returns x such that x^2 is a modulo
    p^n.

    INPUTS:
    a, b, p (integers): p is a prime not dividing a and b^2 is congruent to a
                        modulo p;
    n: a power of p (integer greater than or equal to 2).

    OUTPUT:
    An integer x in 1, 2, ... , p^n - 1 such that x^2 is congruent to a modulo
    p^n.
    """
    if n == 1:
        return b % p
    ...

Let's test it.

In [ ]:
number_of_tests = 30

max_p = 100

min_n = 10
max_n = 20

min_a = 2
max_a = 1000


for _ in range(number_of_tests):
    p = random_prime(max_p, lbound=3)
    n = randint(min_n, max_n)
    R = Zmod(p)
    a = ZZ(randint(min_a, max_a))
    aa = R(a)
    while (not is_square(aa)) or aa == 0:
        a = ZZ(randint(min_a, max_a))
        aa = R(a)
    bb = sqrt(aa)
    b = ZZ(bb)
    x = my_sqrt(a, b, p, n)
    if Mod(x, p^n)^2 != Mod(a, p^n):
        print(f"{x} is not a square root of {a} modulo {p}^{n}.")
        break
else:
    print("It works!")

## Problem 3: Quadratic Reciprocity

In this problem, given an integer $a$ (maybe negative!) and a prime $p$, we check if $a$ is a square modulo $p$ using quadratic reciprocity.

You can simply follow the [algorithm given in the book](https://luisfinotti.org/pcimc/13-Square_Roots.html#al-sqr-qr).

Again, the function `valuation` is useful to remove powers of $2$.

In [ ]:
def my_is_sqr(a, p):
    """
    Use quadratic reciprocity to determine if a is a square modulo p.

    INPUT:
    a: an integer;
    p: a prime.

    OUTPUT:
    True if a is a square modulo p, False if not.
    """
    ...

Let's test it:

In [ ]:
number_of_tests = 30

min_p = 1000
max_p = 2000

for _ in range(number_of_tests):
    p = random_prime(max_p, lbound = min_p)
    a = (-1)^randint(0, 1) * randint(2 , p - 1)
    R = Zmod(p)
    if (is_square(R(a)) != my_is_sqr(a, p)):
        if is_square(R(a)):
            print(f"{a} is a square mod {p}, but your function gives False.")
            break
        else:
            print(f"{a} is not a square mod {p}, but your function gives True.")
            break
else:
    print("It works!")

## Problem 4: Quadratic Sieve (Relation Building)

Given $N = pq$ (for large primes $p$ and $q$), bounds $b$ and $B$ (all positive integers), with $\lfloor \sqrt{N} \rfloor + 1 < b < \lfloor 2\sqrt{N}\rfloor$, you will find all $B$-smooth $x$'s such that $\lfloor \sqrt{N} \rfloor + 1 \leq x \leq b$.

The [algorithm is described in the book](https://luisfinotti.org/pcimc/14-Quad_Sieve_Index_Calc.html#al-rel-build), where you also find [an example](https://luisfinotti.org/pcimc/14-Quad_Sieve_Index_Calc.html#example).

Note that the output must be *two* lists: the $x$'s themselves and the corresponding values of $x^2 - N$ (which are the reductions modulo $N$ of $x^2$)!

**Hints:** 
1) To loop over primes less than or equal to $B$ you can do
    ```python
    # loop over primes between 2 and B (inclusive)
    for p in primes(B + 1):
        ...
    ```
2) To find the first element greater than or equal to `a0` that is congruent to `r` modulo `p^n`, you can do
    ```python
    # first integer >= a0 congruent to r mod p^n
    r0 = r + ceil((a0 - r) / p^n) * p^n
    ```
    This is helpful when finding the first element of `list_a`, with elements from $\lfloor \sqrt{N} \rfloor$ to $b$, congruent to some root `r`, by taking `a0 = list_a[0]`.
3) Since `list_a` (as above) contains consecutive number, the index of an element `x` in `list_a`, assuming that `x` is indeed in the list, is `x - a0` (with `a0 = list_a[0]` again).
4) If `v` and `w` are lists, to create a lists with elements of `v` in positions where `w` has ones, we can do
    ```python
    # elements of v where w is 1
    [v[i] for i, x in enumerate(w) if x == 1]
    ```
5) Use the function `mod_sqrt` below to compute the square roots modulo powers of primes.

<hr />

Part of what you have to do is find all solutions of equations $x^2 \equiv N \pmod{p^n}$.  We could use Problems 1 and 2, but 2 is incomplete, and this part should be independent of the previous ones, so we will use Sage's own routines.

In [ ]:
def mod_sqrt(N, m):
    """
    Returns all (if any) square roots of N modulo m.

    INPUTS:
    N: an integer to take the square root (modulo m);
    m: the modulus.

    OUTPUT:
    A list (perhaps empty) if integers containing all square roots of N modulo
    m.
    """
    R = Zmod(m)
    PR.<x> = PolynomialRing(R)
    f = x^2 - N
    return [ZZ(x) for x in f.roots(multiplicities=False)]

**IMPORTANT:** You cannot use the code of this function to help you with the previous parts!  You must use the methods discussed in the book and explicitly mentioned in the problem.

For instance, to find all roots of $x^2 = 111 \pmod{11^5}$, we do:

In [ ]:
N = 111
p = 11
n = 5
mod_sqrt(N, p^n)

So, two roots, $20756$ and $140295$.

In some cases, there are no roots, like $x^2 \equiv 110 \pmod{11^5}$:

In [ ]:
p = 11
n = 5
N = 110
mod_sqrt(N, p^n)

Modulo $2^n$ we might have more than two roots, like in $x^2 \equiv 113 \pmod{2^7}$:

In [ ]:
p = 2
n = 7
N = 113
mod_sqrt(N, p^n)

In your code, you will have loop over the entries of this list of roots for various $p$ and $n$.

<hr />

In [ ]:
def quad_sieve(N, b, B):
    """
    Apply the quadratic sieve for relation building.  Given an N product of two
    primes, returns the list of elements from floor(sqrt(N)) + 1 to b such that
    their squares reduced modulo N are B-smooth and the list of these B-smooth
    numbers.

    INPUTS:
    N: a product fo two primes;
    b: an upper bound for a's;
    B: upper bound for the prime divisors of the reduction modulo N of the
       a's squared to be returned.

    OUTPUTS:
    A list containing elements from floor(sqrt(N)) + 1 to b such that their
    squares reduced modulo N are B-smooth and the list of these B-smooth
    numbers.
    """
    ...

If you want to test with the example from the book (and class):

In [ ]:
quad_sieve(1073, 46, 19) == ([33, 35, 39, 41, 45], [16, 152, 448, 608, 952])

More tests:

In [ ]:
number_of_tests = 30

min_p = 100
max_p = 200

min_q = 200
max_q = 300

min_B = 20
max_B = 40

for _ in range(number_of_tests):
    p = random_prime(max_p, lbound = min_p)
    q = p
    while q == p:
        q = random_prime(max_q, lbound = min_q)
    N = p * q
    b = floor(sqrt(2 * N)) - 1
    B = randint(min_B, max_B)
    # brute force to compare to sieve
    a0 = floor(sqrt(N)) + 1
    list_c = [x^2 - N for x in range(a0, b + 1)]
    a_expected = []
    c_expected = []
    vprimes = prime_range(B + 1)
    for i, c  in enumerate(list_c):
        w = [p^valuation(c, p) for p in vprimes]
        if c == prod(w):
            a_expected.append(i + a0)
            c_expected.append(c)
    result = quad_sieve(N, b, B)
    if result != (a_expected, c_expected):
        if result[0] != a_expected:
            print(f"Failed.  The a's should be {a_expected}, not {result[0]}.")
        if result[1] != c_expected:
            print(f"Failed.  The c's should be {c_expected}, not {result[1]}.")
        break
else:
    print("It works!")

### Optional: Comparing Times

Let's check how fast the quadratic sieve is in finding lists of smooth numbers.  We will increase the sizes to make more representative of cases with difficult computations.

In [ ]:
min_p = 10000000
max_p = 20000000

min_q = 20000000
max_q = 30000000

min_B = 200
max_B = 400

p = random_prime(max_p, lbound = min_p)
q = p

while q == p:
    q = random_prime(max_q, lbound = min_q)

N = p * q
b = floor(sqrt(2*N)) - 1
B = randint(min_B, max_B)

Fist, we find the smooth numbers by dividing by all prime powers less than $B$ (it can take a few minutes):

In [ ]:
%%time

a0 = floor(sqrt(N)) + 1
list_c = [x^2 - N for x in range(a0, b + 1)]
a_result = []
c_result = []
vprimes = prime_range(B + 1)
for i, c  in enumerate(list_c):
    w = [p^valuation(c, p) for p in vprimes]
    if c == prod(w):
        a_result.append(i + a0)
        c_result.append(c)

res = (a_result, c_result)

Now, using the quadratic sieve:

In [ ]:
%%time
res2 = quad_sieve(N, b, B)

Checking that they are giving us the same results:

In [ ]:
res == res2

If you run those yourself, you will see different times, as the paramenters are random and we are running in different hardware, but the original example I ran gave a difference from 247 seconds to 5 seconds, a very considerable difference.